In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
# read dataset
df=pd.read_csv('Advertising Budget and Sales.csv')
df = df.drop(df.columns[0], axis=1)

# add dates
n_days = df.shape[0]
df['ds'] = pd.date_range(start="2022-01-01", periods=n_days, freq="D")
df['y'] = df['Sales ($)']

In [ ]:
# split train and test
print(df.shape[0])
df_prophet_train = df[df['ds'] < pd.to_datetime('2022-06-01')]
df_prophet_test = df[df['ds'] >= pd.to_datetime('2022-06-01')]
df_prophet_test2 = df_prophet_test.copy()
print(df_prophet_train.shape[0]+df_prophet_test.shape[0])

In [ ]:
# Initialize and fit Prophet model
model = Prophet(yearly_seasonality=True, daily_seasonality=True)
model.add_regressor('TV Ad Budget ($)')
model.add_regressor('Radio Ad Budget ($)')
model.add_regressor('Newspaper Ad Budget ($)')
model.fit(df_prophet_train)

In [ ]:
forecast = model.predict(df_prophet_test)

In [ ]:
result_df = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
result_df['prediction'] = np.where(result_df['yhat'].isnull(), result_df['y'], result_df['yhat'])

In [ ]:
# Plot the data
plt.figure(figsize=(10, 6))  # Set figure size
plt.plot(result_df["ds"], result_df["y"], label="Sales", marker="o", color='k')
plt.plot(result_df["ds"], result_df["yhat"], label="Predicted Sales", marker="s", color='b')

# Add a vertical line for 2024-01-01
highlight_date = pd.Timestamp("2022-06-01")
plt.axvline(x=highlight_date, color="red", linestyle="--", label="2024-01-01")

# Rotate x-axis labels
plt.xticks(rotation=90)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("Values")
plt.title("Line Plot with Rotated Dates")
plt.legend()

# Show the plot
plt.tight_layout()  # Adjust layout to prevent cutoff of labels
plt.show()

In [ ]:
print(result_df[result_df['yhat'].notnull()]['y'].sum())
print(result_df[result_df['yhat'].notnull()]['yhat'].sum())

In [ ]:
available_spend = df_prophet_test['TV Ad Budget ($)'].sum() + df_prophet_test['Radio Ad Budget ($)'].sum() + df_prophet_test['Newspaper Ad Budget ($)'].sum()

print(f"Available Spend: ${available_spend}")

In [ ]:
# import numpy as np
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

budget=10000

def predict_leads(tv_spend, radio_spend, newspaper_spend, model, df_prophet_test):

    nrows=df_prophet_test.shape[0]
    
    # Assign the ad budgets to the dataframe columns
    df_prophet_test['TV Ad Budget ($)'] = [tv_spend/nrows] * nrows 
    df_prophet_test['Radio Ad Budget ($)'] = [radio_spend/nrows] * nrows
    df_prophet_test['Newspaper Ad Budget ($)'] = [newspaper_spend/nrows] * nrows
    
    # Make the forecast using the model
    forecast = model.predict(df_prophet_test)
    
    # Return the sum of the predicted leads (you can use other metrics like the average or max)
    return forecast['yhat'].sum()  # Assuming 'yhat' is the predicted leads

# Define the search space for TV and Radio spend (the Newspaper spend will be calculated from the total budget)
space = [
    Real(0, budget, name='tv_spend'),  # From 0 to 100,000 for TV spend
    Real(0, budget, name='radio_spend')  # From 0 to 100,000 for Radio spend
]

# Load your trained Prophet model (this is a dummy load function, replace it with actual model loading)
prophet_model = model

# Objective function to minimize (negative leads because we want to maximize leads)
@use_named_args(space)
def objective(tv_spend, radio_spend):
    # Calculate the remaining budget for Newspaper spend (total budget is 100,000)
    newspaper_spend = budget - tv_spend - radio_spend
    
    # If the Newspaper spend is negative, return a very low (negative) value
    if newspaper_spend < 0:
        return 0  # This ensures the optimizer doesn't choose invalid combinations
    
    # Get the predicted leads from the Prophet model
    predicted_leads = predict_leads(tv_spend, radio_spend, newspaper_spend, prophet_model, df_prophet_test)
    
    return -predicted_leads  # Minimize the negative of the predicted leads to maximize it

# Run the Bayesian Optimization
result = gp_minimize(objective, space, n_calls=50, random_state=42)

# Output the best results
print("Best TV Spend: ", result.x[0])
print("Best Radio Spend: ", result.x[1])

# Calculate the optimal Newspaper Spend (since the total is budget)
best_newspaper_spend = budget - result.x[0] - result.x[1]
print("Best Newspaper Spend: ", best_newspaper_spend)

# Maximum Predicted Leads
print("Maximum Predicted Leads: ", -result.fun)  # Negate the value to get the actual predicted leads

In [ ]:
df_prophet_test2

In [ ]:
# original
print(result_df['yhat'].sum())

best_view = pd.DataFrame()

tv_spend = 2272
radio_spend = 701
newspaper_spend = 26
nrows = df_prophet_test.shape[0]

# Assign the ad budgets to the dataframe columns
best_view['ds'] = df_prophet_test['ds']
best_view['TV Ad Budget ($)'] = [tv_spend/nrows] * nrows 
best_view['Radio Ad Budget ($)'] = [radio_spend/nrows] * nrows
best_view['Newspaper Ad Budget ($)'] = [newspaper_spend/nrows] * nrows
display(best_view)

# Make the forecast using the model
forecast = model.predict(best_view)

bestview_df = pd.merge(df, forecast[['ds', 'yhat']], on='ds', how='left')
bestview_df['prediction'] = np.where(bestview_df['yhat'].isnull(), bestview_df['y'], bestview_df['yhat'])
display(bestview_df)

# Plot the data
plt.figure(figsize=(10, 6))  # Set figure size
plt.plot(bestview_df["ds"], bestview_df["y"], label="Sales", marker="o", color='k')
plt.plot(bestview_df["ds"], bestview_df["yhat"], label="Predicted Sales", marker="s", color='b')

# Add a vertical line for 2024-01-01
highlight_date = pd.Timestamp("2022-06-01")
plt.axvline(x=highlight_date, color="red", linestyle="--", label="2024-01-01")

# Rotate x-axis labels
plt.xticks(rotation=90)

# Add labels, title, and legend
plt.xlabel("Date")
plt.ylabel("Values")
plt.title("Line Plot with Rotated Dates")
plt.legend()

# Show the plot
plt.tight_layout()  # Adjust layout to prevent cutoff of labels
plt.show()

In [ ]:
bestview_df.yhat.sum()

In [ ]:
# # Define the objective function to optimize the daily spend for TV, Radio, and Newspaper
# def daily_objective(x, day_index):
#     tv_spend, radio_spend, newspaper_spend = x
    
#     # Create a future dataframe with the optimized spend for the current day (day_index)
#     future_optimized = pd.DataFrame({
#         'ds': [df_prophet_test['ds'].iloc[day_index]],  # Only the current day
#         'TV Ad Budget ($)': [tv_spend],
#         'Radio Ad Budget ($)': [radio_spend],
#         'Newspaper Ad Budget ($)': [newspaper_spend]
#     })
    
#     # Predict sales for the current day with the optimized spend
#     predicted_sales = model.predict(future_optimized)['yhat'][0]
    
#     return -predicted_sales  # Minimize negative sales (maximize sales)

# # Optimization for each day in the forecast period
# optimized_spends = []

# # Total available daily spend (can be modified per day or fixed as sum of the budget)
# available_daily_spend = df_prophet_test[['TV Ad Budget ($)', 'Radio Ad Budget ($)', 'Newspaper Ad Budget ($)']].sum(axis=1) / df_prophet_test.shape[0]

# # Run optimization for each day in the test set
# for day_index in range(df_prophet_test.shape[0]):
#     # Constraints: total spend per day should not exceed the available daily spend
#     constraints = (
#         {'type': 'eq', 'fun': lambda x: sum(x) - available_daily_spend.iloc[day_index]}  # Total spend constraint for the day
#     )
    
#     # Bounds for each variable (spend must be non-negative)
#     bounds = [(0, available_daily_spend.iloc[day_index])] * 3  # TV, Radio, and Newspaper spend
    
#     # Initial guess for the optimization (evenly split available spend for the day)
#     initial_guess = [available_daily_spend.iloc[day_index] / 3] * 3

#     # Run the optimization for the current day
#     result = minimize(daily_objective, initial_guess, args=(day_index,), bounds=bounds, constraints=constraints)
    
#     # Store the optimized spend allocation for the current day
#     optimized_spends.append(result.x)

# # Convert the optimized spend allocations to a DataFrame
# optimized_spends_df = pd.DataFrame(optimized_spends, columns=['TV Ad Spend ($)', 'Radio Ad Spend ($)', 'Newspaper Ad Spend ($)'])
# optimized_spends_df['Date'] = df_prophet_test['ds'].reset_index(drop=True)

# # Display the optimized spends for each day
# print(optimized_spends_df.head())

# # Plot the optimized spend allocations for each channel
# plt.figure(figsize=(10, 6))
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['TV Ad Spend ($)'], label="Optimized TV Spend", color='b')
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['Radio Ad Spend ($)'], label="Optimized Radio Spend", color='g')
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['Newspaper Ad Spend ($)'], label="Optimized Newspaper Spend", color='r')
# plt.xlabel("Date")
# plt.ylabel("Optimized Spend ($)")
# plt.title("Optimized Daily Spend Allocation Across Channels")
# plt.legend()
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()

In [ ]:
# # Define the objective function to optimize the daily spend for TV, Radio, and Newspaper
# def daily_objective(x, day_index):
#     tv_spend, radio_spend, newspaper_spend = x
    
#     # Create a future dataframe with the optimized spend for the current day (day_index)
#     future_optimized = pd.DataFrame({
#         'ds': [df_prophet_test['ds'].iloc[day_index]],  # Only the current day
#         'TV Ad Budget ($)': [tv_spend],
#         'Radio Ad Budget ($)': [radio_spend],
#         'Newspaper Ad Budget ($)': [newspaper_spend]
#     })
    
#     # Predict sales for the current day with the optimized spend
#     predicted_sales = model.predict(future_optimized)['yhat'][0]
    
#     return -predicted_sales  # Minimize negative sales (maximize sales)

# # Optimization for each day in the forecast period
# optimized_spends = []

# # Total available daily spend (can be modified per day or fixed as sum of the budget)
# available_daily_spend = df_prophet_test[['TV Ad Budget ($)', 'Radio Ad Budget ($)', 'Newspaper Ad Budget ($)']].sum(axis=1) / df_prophet_test.shape[0]

# # Run optimization for each day in the test set
# for day_index in range(df_prophet_test.shape[0]):
#     # Constraints: total spend per day should not exceed the available daily spend
#     constraints = (
#         {'type': 'eq', 'fun': lambda x: sum(x) - available_daily_spend.iloc[day_index]}  # Total spend constraint for the day
#     )
    
#     # Bounds for each variable (spend must be non-negative)
#     bounds = [(0, available_daily_spend.iloc[day_index])] * 3  # TV, Radio, and Newspaper spend
    
#     # Initial guess for the optimization (evenly split available spend for the day)
#     initial_guess = [available_daily_spend.iloc[day_index] / 3] * 3

#     # Run the optimization for the current day
#     result = minimize(daily_objective, initial_guess, args=(day_index,), bounds=bounds, constraints=constraints)
    
#     # Store the optimized spend allocation for the current day
#     optimized_spends.append(result.x)

# # Convert the optimized spend allocations to a DataFrame
# optimized_spends_df = pd.DataFrame(optimized_spends, columns=['TV Ad Spend ($)', 'Radio Ad Spend ($)', 'Newspaper Ad Spend ($)'])
# optimized_spends_df['Date'] = df_prophet_test['ds'].reset_index(drop=True)

# # Display the optimized spends for each day
# print(optimized_spends_df.head())

# # Now, let's calculate the extra leads:

# # Create a dataframe for the original spend allocations (before optimization)
# original_spends_df = df_prophet_test[['ds', 'TV Ad Budget ($)', 'Radio Ad Budget ($)', 'Newspaper Ad Budget ($)']]

# # Calculate predicted sales with the original spend allocations
# future_original = df_prophet_test[['ds', 'TV Ad Budget ($)', 'Radio Ad Budget ($)', 'Newspaper Ad Budget ($)']].copy()
# forecast_original = model.predict(future_original)

# # Calculate predicted sales with the optimized spend allocations
# optimized_spends_df_copy = optimized_spends_df[['Date', 'TV Ad Spend ($)', 'Radio Ad Spend ($)', 'Newspaper Ad Spend ($)']].copy()
# optimized_spends_df_copy.columns = ['ds', 'TV Ad Budget ($)', 'Radio Ad Budget ($)', 'Newspaper Ad Budget ($)']
# forecast_optimized = model.predict(optimized_spends_df_copy)

# # Calculate the extra leads (sales) by subtracting the original predicted sales from the optimized predicted sales
# extra_leads = forecast_optimized['yhat'] - forecast_original['yhat']

# # Add the extra leads to the optimized spends dataframe
# optimized_spends_df['Extra Leads'] = extra_leads

# # Display the extra leads along with the optimized spend allocations
# print(optimized_spends_df[['Date', 'TV Ad Spend ($)', 'Radio Ad Spend ($)', 'Newspaper Ad Spend ($)', 'Extra Leads']].head())

# # Plot the optimized spend allocations for each channel
# plt.figure(figsize=(10, 6))
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['TV Ad Spend ($)'], label="Optimized TV Spend", color='b')
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['Radio Ad Spend ($)'], label="Optimized Radio Spend", color='g')
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['Newspaper Ad Spend ($)'], label="Optimized Newspaper Spend", color='r')
# plt.xlabel("Date")
# plt.ylabel("Optimized Spend ($)")
# plt.title("Optimized Daily Spend Allocation Across Channels")
# plt.legend()
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()

# # Plot the extra leads generated by the optimized spend allocations
# plt.figure(figsize=(10, 6))
# plt.plot(optimized_spends_df['Date'], optimized_spends_df['Extra Leads'], label="Extra Leads from Optimization", color='purple')
# plt.xlabel("Date")
# plt.ylabel("Extra Leads")
# plt.title("Extra Leads from Optimized Spend Allocations")
# plt.legend()
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()